##Encoding Text

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

###Bag of Words
 This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency.NO sequence here

In [ ]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1}
Negative: {1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 13: 1, 10: 1, 11: 1, 12: 1, 9: 1}


###Integer Encoding
The next technique we will look at is called **integer encoding**. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.


In [ ]:
vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 5, 12, 13]
Negative: [1, 2, 3, 4, 5, 6, 7, 8, 13, 10, 11, 5, 12, 9]


###Word Embeddings
Luckily there is a third method that is far superior, **word embeddings**. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.


##Recurrent Neural Networks (RNN's)
 All our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exaclty what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.
![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Let's define what all these variables stand for before we get into the explination.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

##Sentiment Analysis


*the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.*

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)
train_data[1]
#ADD PADDING TO REVIEW TO MAKE IT EQUAL 250 WORDS
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 2s 0us/step


# MODEL
bedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer.

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
#sigmoid bec more than 0.5 consider as positive review while less consider as negative review
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)
results = model.evaluate(test_data, test_labels)
print(results)

Epoch 1/10
625/625 [==============================] - 49s 73ms/step - loss: 0.4522 - acc: 0.7743 - val_loss: 0.4837 - val_acc: 0.8306
Epoch 2/10
625/625 [==============================] - 20s 31ms/step - loss: 0.2591 - acc: 0.8995 - val_loss: 0.2881 - val_acc: 0.8824
Epoch 3/10
625/625 [==============================] - 12s 20ms/step - loss: 0.2000 - acc: 0.9269 - val_loss: 0.2963 - val_acc: 0.8784
Epoch 4/10
625/625 [==============================] - 13s 21ms/step - loss: 0.1625 - acc: 0.9423 - val_loss: 0.3168 - val_acc: 0.8850
Epoch 5/10
625/625 [==============================] - 10s 17ms/step - loss: 0.1328 - acc: 0.9534 - val_loss: 0.3825 - val_acc: 0.8766
Epoch 6/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1123 - acc: 0.9627 - val_loss: 0.3246 - val_acc: 0.8764
Epoch 7/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0911 - acc: 0.9704 - val_loss: 0.3968 - val_acc: 0.8656
Epoch 8/10
625/625 [==============================] - 9s 15ms/st

In [5]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

1641221/1641221 [==============================] - 1s 1us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [6]:

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was good! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really good."
predict(negative_review)

1/1 [==============================] - 0s 350ms/step
[0.9795592]
1/1 [==============================] - 0s 19ms/step
[0.6932592]


##RNN Play Generator


In [22]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# to load your own
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
print(text[:250])
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [24]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))


Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]
First Citizen


In [25]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [26]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_3 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape
print(len(example_batch_predictions))
print(example_batch_predictions)
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

(64, 100, 65) # (batch_size, sequence_length, vocab_size)
64
tf.Tensor(
[[[ 1.02750876e-03  1.53488829e-03  9.09130438e-04 ... -5.22715505e-04
    4.44042357e-03 -1.59466779e-03]
  [ 5.36487671e-03 -6.60190312e-03 -2.90446356e-03 ... -1.52667263e-03
    8.11385689e-04 -8.50743148e-04]
  [ 7.20796362e-03 -5.18095167e-03  7.30615226e-04 ...  5.53085236e-04
   -1.34372106e-03  2.00896012e-03]
  ...
  [ 2.57946295e-03  2.60446384e-03  1.22565553e-02 ... -1.25965029e-02
    9.66318417e-03 -1.41316652e-03]
  [-3.30607360e-03 -1.10397302e-03  8.33036657e-03 ... -1.27344774e-02
    1.07694948e-02 -2.73559382e-03]
  [ 4.02823091e-03 -6.22986350e-04  2.51719635e-03 ... -1.39741134e-02
    5.14244195e-03 -1.82784686e-03]]

 [[-2.09902273e-03 -4.04894538e-03  1.92816975e-03 ...  8.76022459e-05
   -5.01998980e-03  1.50309736e-03]
  [-8.96278885e-04 -6.50093891e-03  5.05822711e-04 ...  7.63851870e-03
   -1.20211681e-02  6.33688085e-03]
  [ 3.60967894e-03 -5.10870898e-03  3.74075631e-03 ...  7.672762

In [28]:
model.compile(optimizer='adam', loss=loss)
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [29]:
history = model.fit(data, epochs=60, callbacks=[checkpoint_callback])

Epoch 1/60
172/172 [==============================] - 15s 65ms/step - loss: 2.5869
Epoch 2/60
172/172 [==============================] - 14s 64ms/step - loss: 1.8824
Epoch 3/60
172/172 [==============================] - 13s 65ms/step - loss: 1.6341
Epoch 4/60
172/172 [==============================] - 13s 66ms/step - loss: 1.5011
Epoch 5/60
172/172 [==============================] - 13s 67ms/step - loss: 1.4208
Epoch 6/60
172/172 [==============================] - 14s 67ms/step - loss: 1.3642
Epoch 7/60
172/172 [==============================] - 14s 67ms/step - loss: 1.3187
Epoch 8/60
172/172 [==============================] - 13s 68ms/step - loss: 1.2805
Epoch 9/60
172/172 [==============================] - 13s 68ms/step - loss: 1.2430
Epoch 10/60
172/172 [==============================] - 13s 68ms/step - loss: 1.2075
Epoch 11/60
172/172 [==============================] - 14s 68ms/step - loss: 1.1714
Epoch 12/60
172/172 [==============================] - 13s 68ms/step - loss: 1.1345
E

In [30]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [31]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

In [34]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: There is nothing either good or bad, but thinking makes it so. 
There is nothing either good or bad, but thinking makes it so. I do you know
My daughters all up And look us with an arby?

Post:
Well, Signior Peter! Can you not speak! all devise
VOLUMNIA:
Indeed, you shall not. He says he so bright at die they dream;
Then weep nor from a pitchel changeance, as thou dost of those
butthe contented.

VOLUMNIA:
Too late, I cannot do't:
Lest I not be a lang'd, in God's spirit all;
And in this trantful ears that with his friend,
To choose me for that judgment, and entreat me faint!
a whoreson, for I know not how to show their friends.
Your laster VINCENTIO:
The thice of the univerying darks
Which makes me swear the with the breath of penitence:
pulchastisement be pardon.

ISABELLA:
To-morrow; you lie in briff: therefore
weeping doth depart and shrewd are gone.

LEONTES:
Ho! first since mine enemies?

QUEEN MARGARET:
Forb
